In [1]:
import tensorflow as tf
import numpy as np
import torch
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from conversion_tf import GEMMDecisionTreeImpl
from hummingbird.ml import convert

2023-03-06 23:15:23.404690: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
BATCH_SIZE = 512
N_FEATURES = 4
N_TREES = 1
MAX_DEPTH = 1

In [3]:
x = tf.constant([5, 5, 5, 5, 5, 5, 5, 5], shape=[1,8])
y = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1,8])

2023-03-06 23:15:25.584325: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 23:15:25.584978: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
forest = RandomForestClassifier(n_estimators=N_TREES, max_depth=MAX_DEPTH)

In [5]:
def representative_dataset():
    for _ in range(100):
      data = np.random.uniform(low=0., high=10., size=(BATCH_SIZE, N_FEATURES))
      yield [data.astype(np.float32)]
 
tf.config.run_functions_eagerly(True)

X, y = make_classification(n_samples=1300, n_features=N_FEATURES,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=True,
                           n_classes=2)

x_train, y_train = X[:1000], y[:1000]
x_test, y_test = X[1000:], y[1000:]

forest.fit(x_train, y_train)

RandomForestClassifier(max_depth=1, n_estimators=1)

### inputs

In [6]:
# 2 features
X_2 = tf.constant([1, 2], shape=[1, 2], dtype=tf.int8)
X_2_f = tf.cast(X_2, tf.float32)

X_f_b = np.random.randint(low=0, high=10, size=(BATCH_SIZE, N_FEATURES)).astype(np.int8)
X_f_b_f = X_f_b.astype(np.float32)

# 8 features
X = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1, 8], dtype=tf.int32)
X_float = tf.constant([1., 2., 3., 4., 5., 6., 7., 8.], shape=[1, 8])
X_8 = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1, 8], dtype=tf.int8)

X_300 = np.random.randint(low=0, high=10, size=(300, 8)).astype(np.int8)
X_300_f = X_300.astype(np.float32)

X_1k_f = np.random.randint(low=0, high=10, size=(1000, 8)).astype(np.float32)
X_1k = X_1k_f.astype(np.int8)


In [7]:
conv_model = convert(forest, 'torch', extra_config={"tree_implementation":"gemm"})

In [8]:
op = conv_model.model._operators[0]

In [9]:
model_gemm = GEMMDecisionTreeImpl(forest)

In [10]:
y_gemm = model_gemm(X_f_b_f)

In [11]:
y_gemm

<tf.Tensor: shape=(512, 2), dtype=float32, numpy=
array([[0.13651878, 0.8634812 ],
       [0.13651878, 0.8634812 ],
       [0.13651878, 0.8634812 ],
       ...,
       [0.13651878, 0.8634812 ],
       [0.13651878, 0.8634812 ],
       [0.13651878, 0.8634812 ]], dtype=float32)>

In [12]:
concrete_func = model_gemm.__call__.get_concrete_function()

In [13]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], model_gemm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  
converter.inference_output_type = tf.int8

In [14]:
tflite_model_gemm = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpbjd043fn/assets
Estimated count of arithmetic ops: 0.044 M  ops, equivalently 0.022 M  MACs
Estimated count of arithmetic ops: 0.044 M  ops, equivalently 0.022 M  MACs


2023-03-06 23:15:26.575350: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-06 23:15:26.575371: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-06 23:15:26.575862: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpbjd043fn
2023-03-06 23:15:26.576118: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-06 23:15:26.576129: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpbjd043fn
2023-03-06 23:15:26.576263: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-03-06 23:15:26.576769: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-03-06 23:15:26.576972: I tensorflow/cc/save

In [15]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_gemm)
interpreter.allocate_tensors()

output = interpreter.get_output_details()[0]  
input = interpreter.get_input_details()[0]  

interpreter.set_tensor(input['index'], X_f_b)
interpreter.invoke()
y_lite_gemm = interpreter.get_tensor(output['index'])

y_pred_lite_gemm = np.argmax(y_lite_gemm, axis=1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [16]:
y_gemm

<tf.Tensor: shape=(512, 2), dtype=float32, numpy=
array([[0.13651878, 0.8634812 ],
       [0.13651878, 0.8634812 ],
       [0.13651878, 0.8634812 ],
       ...,
       [0.13651878, 0.8634812 ],
       [0.13651878, 0.8634812 ],
       [0.13651878, 0.8634812 ]], dtype=float32)>

In [17]:
y_lite_gemm

array([[-87, 127],
       [-87, 127],
       [-87, 127],
       ...,
       [-87, 127],
       [-87, 127],
       [-87, 127]], dtype=int8)

### naming model_abcd
a = number of trees <br>
b = max_depth <br>
c = #features <br>
d = batch size <br>

In [18]:
with open(f'../saved_models/random_forest/gemm/float32/final_eval/model_{N_TREES}_{MAX_DEPTH}_{N_FEATURES}_{BATCH_SIZE}.tflite', 'wb') as f:
    f.write(tflite_model_gemm)

# Current stats
### GEMM model Edge TPU
name | value |
|:---------|:---------|
Input model| model.tflite 
Input size | 4.54MiB 
Output model | model_edgetpu.tflite 
Output size | 20.48MiB 
On-chip memory used for caching model parameters | 7.67MiB 
On-chip memory remaining for caching model parameters | 34.25KiB 
Off-chip memory used for streaming uncached model parameters | 9.37MiB 
Number of Edge TPU subgraphs | 4 
Total number of operations | 437 
Number of operations that will run on Edge TPU | 427 
Number of operations that will run on CPU | 10
